In [1]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [2]:
def EvenDifference(points):
    """
    Calculates the finite difference table for evenly spaced nodes.
    
    Args:
        points (list of tuples): A list of (x, y) data points, sorted by x.

    Returns:
        pandas.DataFrame: The full difference table.
    """
    x_values = [p[0] for p in points]
    y_values = [p[1] for p in points]
    n = len(y_values)

    # Internal calculation table
    diff_calc_table = np.full((n, n), np.nan)
    diff_calc_table[:, 0] = y_values
    for j in range(1, n):
        for i in range(n - j):
            diff_calc_table[i, j] = diff_calc_table[i+1, j-1] - diff_calc_table[i, j-1]

    # Format the output DataFrame
    data = {
        'x_i': x_values,
        'y_i': y_values
    }
    for j in range(1, n):
        # Pad with NaN to maintain table shape
        col_name = f'Order {j}'
        col_data = np.full(n, np.nan)
        col_data[:(n-j)] = diff_calc_table[:(n-j), j]
        data[col_name] = col_data
    
    df = pd.DataFrame(data)
    return df


# Gauss II

## Algorithm

0. Conditions
* Used for **evenly spaced nodes**: $x_k = x_0 + kh$, $k \in \mathbb{Z}$.
* Requires an **odd number of nodes** (which means an even degree $n$).
* The formula originates from the **central node** $x_0$.

1. Finite Difference Table

Given $n+1$ nodes (where $n$ is even), with $x_0$ as the central node.
A standard finite difference table is computed.

*(This is performed by the `EvenDifference(points)` function, which can be found in `Week6-Interpolation/pp2_Newton/pp2b_NewtonFixedGap.ipynb`)*

2. Coefficient Selection

The coefficients $C_i$ are selected from the difference table in a "zig-zag" path (Left-Right) starting from $y_0$:
* $C_0 = y_0$
* $C_1 = \Delta y_{-1}$
* $C_2 = \Delta^2 y_{-1}$
* $C_3 = \Delta^3 y_{-2}$
* $C_4 = \Delta^4 y_{-2}$
* $C_5 = \Delta^5 y_{-3}$
* ...
* **General form:**
    * $C_{2i} = \Delta^{2i} y_{-i}$
    * $C_{2i-1} = \Delta^{2i-1} y_{-i}$ (for $i \ge 1$)

3. Polynomial Construction in $t$

The polynomial is constructed in the variable $t = \frac{x - x_0}{h}$:
$P(t) = \sum_{i=0}^{n} D_i B_i(t)$

1.  **Calculate Main Coefficients $D_i$:**
    $D_i = \frac{C_i}{i!}$

2.  **Calculate Basis Polynomials $B_i(t)$:**
    * $B_0(t) = 1$
    * $B_1(t) = B_0(t) \cdot t = t$
    * $B_2(t) = B_1(t) \cdot (t + 1)$
    * $B_3(t) = B_2(t) \cdot (t - 1)$
    * $B_4(t) = B_3(t) \cdot (t + 2)$
    * ...
    * **General form:**
        * $B_{2k}(t) = B_{2k-1}(t) \cdot (t + k)$
        * $B_{2k+1}(t) = B_{2k}(t) \cdot (t - k)$

3.  **Compute Total Polynomial $P(t)$:**
    $P(t) = \sum_{i=0}^{n} N_i(t) = \sum_{i=0}^{n} D_i B_i(t)$
    * The coefficients of each $N_i(t)$ are summed by degree to find the final coefficients of $P(t) = a_0 + a_1 t + \dots + a_n t^n$.

4.  **Output:** Return the intermediate steps table and the final coefficient list $a_k$ for $P(t)$.

In [3]:
def gauss_2_interpolation(points, x0_index):
    """
    Constructs the Gauss II interpolation polynomial in the variable t = (x - x0) / h.
    
    Args:
        points (list of tuples): List of (x, y) data points. Must be evenly spaced
                                 and have an odd number of points.
        x0_index (int): The index of the central node (x0).

    Returns:
        step_pd (pd.DataFrame): DataFrame showing the intermediate steps.
        coeff_pd (pd.DataFrame): DataFrame of the final polynomial coefficients for P(t).
    """
    
    n = len(points) - 1
    if (n + 1) % 2 == 0:
        raise ValueError("Gauss II formula requires an odd number of nodes (even degree n).")
    
    # --- 1. Generate Difference Table ---
    # We use the re-usable function from pp2b_NewtonFixedGap.ipynb
    diff_table_df = EvenDifference(points)

    # --- 2. Select Coefficients (Gauss II Path) ---
    C_coeffs = []
    for i in range(n + 1):
        order_col = f'Order {i}' if i > 0 else 'y_i'
        
        # This is the selection rule for Gauss II
        # y0, y-1, y-1, y-2, y-2, ...
        row_index = x0_index - (i + 1) // 2
        
        if row_index < 0 or row_index >= len(points):
             raise IndexError(f"Cannot access index {row_index} in difference table for order {i}.")
             
        C_coeffs.append(diff_table_df[order_col].iloc[row_index])

    # --- 3. Build Polynomial P(t) ---
    steps_data = []
    N_coeffs_total = np.zeros(n + 1, dtype=float)
    B_coeffs = np.array([1.0])  # B_0(t) = 1

    for i in range(n + 1):
        D_i = C_coeffs[i] / math.factorial(i)
        
        # Calculate B_i(t) from B_{i-1}(t)
        if i == 1:
            # B_1(t) = t
            B_coeffs = np.array([0.0, 1.0]) # [const, t]
        elif i > 1:
            if i % 2 == 0: # Even: B_2k = B_{2k-1} * (t + k)
                k = i // 2
                B_coeffs = np.convolve(B_coeffs, [k, 1])
            else: # Odd: B_{2k+1} = B_{2k} * (t - k)
                k = (i - 1) // 2
                B_coeffs = np.convolve(B_coeffs, [-k, 1])
        
        # N_i(t) = D_i * B_i(t)
        Ni_coeffs = D_i * B_coeffs
        
        # Add to total polynomial (pad with zeros)
        N_coeffs_total[:len(Ni_coeffs)] += Ni_coeffs

        # Store intermediate steps for printing
        steps_data.append({
            'i': i,
            'Difference Coeff': C_coeffs[i],
            'D_i = C_i / i!': D_i,
            'B_i(t) Coeffs (low->high)': B_coeffs.tolist(),
            'N_i(t) Coeffs (low->high)': Ni_coeffs.tolist()
        })

    step_pd = pd.DataFrame(steps_data)
    coeff_pd = pd.DataFrame({
        'Degree (t)': np.arange(n + 1),
        'Coeff': N_coeffs_total
    })

    return step_pd, coeff_pd

## Result

In [4]:
# 1. Define the data points from the image
points = [
    (9.2, 9.4341319),
    (9.3, 9.4307764),
    (9.4, 9.4261142),
    (9.5, 9.4211191),
    (9.6, 9.4170553),
    (9.7, 9.4147476),
    (9.8, 9.4152900),
    (9.9, 9.4196762),
    (10.0, 9.4288617)
]

# 2. Set the central node index
# 9 points (indices 0-8), the center is index 4
x0_index = 4
x0_val = points[x0_index][0]
h = points[1][0] - points[0][0]

In [5]:
print("--- Generated Finite Difference Table ---")
df = EvenDifference(points)

df.style

--- Generated Finite Difference Table ---


,x_i,y_i,Order 1,Order 2,Order 3,Order 4,Order 5,Order 6,Order 7,Order 8
0,9.200000,9.434132,-0.003356,-0.001307,0.000974,0.000290,-0.000730,0.001438,-0.002516,0.004026
1,9.300000,9.430776,-0.004662,-0.000333,0.001264,-0.000439,0.000709,-0.001078,0.001510,nan
2,9.400000,9.426114,-0.004995,0.000931,0.000825,0.000269,-0.000369,0.000432,nan,nan
3,9.500000,9.421119,-0.004064,0.001756,0.001094,-0.000100,0.000062,nan,nan,nan
4,9.600000,9.417055,-0.002308,0.002850,0.000994,-0.000038,nan,nan,nan,nan
5,9.700000,9.414748,0.000542,0.003844,0.000956,nan,nan,nan,nan,nan
6,9.800000,9.415290,0.004386,0.004799,nan,nan,nan,nan,nan,nan
7,9.900000,9.419676,0.009186,nan,nan,nan,nan,nan,nan,nan
8,10.000000,9.428862,nan,nan,nan,nan,nan,nan,nan,nan


In [6]:
# 3. Calculate the Gauss II polynomial
step_df, final_coeff_df = gauss_2_interpolation(points, x0_index)

print("\n--- Polynomial Construction Steps (Gauss II) ---")
step_df.style


--- Polynomial Construction Steps (Gauss II) ---


,i,Difference Coeff,D_i = C_i / i!,B_i(t) Coeffs (low->high),N_i(t) Coeffs (low->high)
0,0,9.417055,9.417055,[1.0],[9.4170553]
1,1,-0.004064,-0.004064,"[0.0, 1.0]","[-0.0, -0.004063800000000839]"
2,2,0.001756,0.000878,"[0.0, 1.0, 1.0]","[0.0, 0.0008780500000007407, 0.0008780500000007407]"
3,3,0.000825,0.000137,"[0.0, -1.0, 0.0, 1.0]","[0.0, -0.0001374666666669929, 0.0, 0.0001374666666669929]"
4,4,0.000269,0.000011,"[0.0, -2.0, -1.0, 2.0, 1.0]","[0.0, -2.2433333333034728e-05, -1.1216666666517364e-05, 2.2433333333034728e-05, 1.1216666666517364e-05]"
5,5,0.000709,0.000006,"[0.0, 4.0, 0.0, -5.0, 0.0, 1.0]","[0.0, 2.3619999999861345e-05, 0.0, -2.9524999999826683e-05, 0.0, 5.904999999965336e-06]"
6,6,-0.001078,-0.000001,"[0.0, 12.0, 4.0, -15.0, -5.0, 3.0, 1.0]","[-0.0, -1.796833333319962e-05, -5.989444444399873e-06, 2.2460416666499523e-05, 7.486805555499841e-06, -4.492083333299905e-06, -1.4973611110999682e-06]"
7,7,-0.002516,-0.000000,"[0.0, -36.0, 0.0, 49.0, 0.0, -14.0, 0.0, 1.0]","[-0.0, 1.797499999995265e-05, -0.0, -2.4465972222157775e-05, -0.0, 6.990277777759364e-06, -0.0, -4.993055555542403e-07]"
8,8,0.004026,0.000000,"[0.0, -144.0, -36.0, 196.0, 49.0, -56.0, -14.0, 4.0, 1.0]","[0.0, -1.4379285714235683e-05, -3.5948214285589207e-06, 1.9571805555487458e-05, 4.892951388871864e-06, -5.591944444424987e-06, -1.3979861111062469e-06, 3.994246031732134e-07, 9.985615079330335e-08]"


In [7]:
print("\n--- Final Polynomial P(t) Coefficients ---")
print(f"P(t) = a0 + a1*t + ... (where t = (x - {x0_val:.1f}) / {h:.1f})")
final_coeff_df.style


--- Final Polynomial P(t) Coefficients ---
P(t) = a0 + a1*t + ... (where t = (x - 9.6) / 0.1)


,Degree (t),Coeff
0,0,9.417055
1,1,-0.003336
2,2,0.000857
3,3,0.000148
4,4,0.000024
5,5,0.000003
6,6,-0.000003
7,7,-0.000000
8,8,0.000000


## Further Test

In [8]:
#Horner Test
def synthetic_division(a, c):
    """
    Perform synthetic division for polynomial p(x) with coefficients a,
    evaluated at x = c.

    Parameters:
        a (list[float]): coefficients of p(x) from highest to lowest degree
        c (float): the value to evaluate p(c)

    Returns:
        df (pd.DataFrame): table with columns [i, a_i, b_i*c, b_i]
        p_c (float): value of p(c)
        q_coeff (list[float]): coefficients of q(x) = (p(x) - p(c)) / (x - c)

        Actual differentation P'(k) (x) = P'(k) (c) / h^k
    """

    n = len(a) - 1
    b = [0.0] * (n + 1)
    bc_values = [""] * (n + 1)

    b[n] = a[n]
    for i in range(n - 1, -1, -1):
        b[i] = a[i] + c * b[i + 1]
        bc_values[i + 1] = b[i + 1] * c

    # Prepare table (i from n to 0)
    df = pd.DataFrame({
        "i": list(range(n, -1, -1)),
        "a_i": [a[i] for i in range(n, -1, -1)],
        "b_i*c": [bc_values[i] for i in range(n, -1, -1)],
        "b_i = a_i + b_(i+1)*c": [b[i] for i in range(n, -1, -1)]
    })

    p_c = b[0]
    q_coeff = b[1:]
    return df, p_c, q_coeff, b

def all_derivatives(a, c, gap):
    """
    Compute all derivatives p^(i)(c) using repeated Horner division
    and display in transposed table format.
    """
    coeffs = a.copy()
    degree = len(a) - 1
    results = []
    b0_list = []
    derivative_list = []
    actual_dev_list = []

    # Perform repeated synthetic division
    for i in range(degree + 1):
        df, b0, next_coeff, b_all = synthetic_division(coeffs, c)
        results.append(b_all)
        b0_list.append(b0)
        derivative_list.append(b0 * math.factorial(i))
        actual_dev_list.append(b0 * math.factorial(i) / gap**i)
        coeffs = next_coeff
        if len(coeffs) == 0:
            break

    # Pad b_i lists for equal column length
    max_len = max(len(b) for b in results)
    for b in results:
        b.extend([None] * (max_len - len(b)))

    # Create DataFrame horizontally
    df = pd.DataFrame(results).T
    df.columns = [f"i={i}" for i in range(len(results))]

    # Insert first column for original a coefficients
    a_col = a + [None] * (df.shape[0] - len(a))
    df.insert(0, "a_i", a_col)

    # Add b_0 and p^(i)(c) rows
    df.loc["b_0"] = [None] + b0_list
    df.loc["p^(i)(c)"] = [None] + derivative_list
    df.loc["p^(i)(x)"] = [None] + actual_dev_list

    # Add a row on top showing the value of c
    df.loc["c"] = [c] + [None] * (df.shape[1] - 1)
    df = df.loc[["c"] + [idx for idx in df.index if idx != "c"]]  # Move row to top

    return df

In [9]:
coeff_list = final_coeff_df['Coeff'].tolist()

x_val = 9.68
t_val = (x_val - x0_val) / h

print(f"x0_val: {x0_val}; gap: {h}")

df2 = all_derivatives(coeff_list, t_val, h)
df2.style

x0_val: 9.6; gap: 0.10000000000000142


,a_i,i=0,i=1,i=2,i=3,i=4,i=5,i=6,i=7,i=8
c,0.800000,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,9.417055,9.415020,-0.001632,0.001300,0.000212,0.000008,-0.000010,-0.000002,0.000001,0.000000
1,-0.003336,-0.002544,0.001139,0.000201,0.000014,-0.000008,-0.000002,0.000000,0.000000,nan
2,0.000857,0.000991,0.000185,0.000019,-0.000006,-0.000002,0.000000,0.000000,nan,nan
3,0.000148,0.000167,0.000023,-0.000004,-0.000003,0.000000,0.000000,nan,nan,nan
4,0.000024,0.000024,-0.000002,-0.000003,0.000000,0.000000,nan,nan,nan,nan
5,0.000003,0.000000,-0.000003,0.000000,0.000000,nan,nan,nan,nan,nan
6,-0.000003,-0.000003,0.000000,0.000000,nan,nan,nan,nan,nan,nan
7,-0.000000,-0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
8,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan,nan
